In [4]:
import conceptnet_lite
conceptnet_lite.connect("../OPENAI/data/conceptnet.db")
from conceptnet_lite import Label, edges_for, edges_between

In [5]:
relaciones_generales=["is_a","etymologically_related_to","manner_of","has_a","derived_from","has_property","form_of","causes","has_prerequisite","has_subevent","has_first_subevent","relate_to","similar_to​"]
relaciones_especificas=["is_a","manner_of","has_a","derived_from","has_property","form_of","causes","has_prerequisite","has_subevent","has_first_subevent"]

In [6]:
def bag_of_synonyms(word):
    sinonimos=set()
    try:
        for e in edges_for(Label.get(text=word, language='en').concepts, same_language=True):
            if e.relation.name == "synonym":
                if word== e.start.text:
                    sinonimos.add(e.end.text)
                elif word== e.end.text:
                    sinonimos.add(e.start.text)
    except:
        pass
    sinonimos.add(word)
    return sinonimos

def bag_of_antonyms(word):
    antonimos=set()
    try:
        for e in edges_for(Label.get(text=word, language='en').concepts, same_language=True):
            if e.relation.name in ["antonym","distinc_from"]:
                if word== e.start.text:
                    antonimos.add(e.end.text)
                elif word== e.end.text:
                    antonimos.add(e.start.text)
    except:
        pass
    return antonimos

def bag_of_hyperonyms(word):
    hiperonimos=set()
    try:
        for e in edges_for(Label.get(text=word, language='en').concepts, same_language=True):
            if e.relation.name in relaciones_generales:
                if word== e.start.text:
                    hiperonimos.add(e.end.text)
    except:
        pass
    return hiperonimos

def bag_of_hyponyms(word):
    hiponimos=set()
    try:
        for e in edges_for(Label.get(text=word, language='en').concepts, same_language=True):
            if e.relation.name in relaciones_especificas:
                if word== e.end.text:
                    hiponimos.add(e.start.text)
                    print(e.relation.name,e.start.text)
    except:
        pass
    return hiponimos


In [7]:
bag_of_hyponyms("brother_in_law")

derived_from co_brother_in_law
form_of brothers_in_law
is_a co_brother
derived_from co_brother_in_law
derived_from half_brother_in_law


{'brothers_in_law', 'co_brother', 'co_brother_in_law', 'half_brother_in_law'}

In [8]:
def jaro_distanceDavid(s1, s2,sinT,sinH,antT,antH,HipT,HipH,hipH) :
    bandera=True

    # Length of two strings
    len1 = len(s1)
    len2 = len(s2)

    # If the listas de tokens are equal 
    if len1==len2:
        for i in range(len1):
            if s1[i]!=s2[i]:
                bandera=False
                break
        if (bandera):
            return 1.0,0.0; 
 
    if (len1 == 0 or len2 == 0) :
        return 0.0,0.0,0.0; 
 
    # Maximum distance upto which matching 
    # is allowed 
    max_dist = (max(len(s1), len(s2)) // 2 )-1 ; 
 
    # Count of matches 
    match = 0; 
    matchC = 0; 
 
    # Hash for matches 
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)
 
    # Traverse through the first string 
    for i in range(len1) : 
            
        # Check if there is any matches 
        for j in range( max(0, i - max_dist),
                    min(len2, i + max_dist + 1)) : 
            #print(s1[i],s2[j])
            # If there is a match or is contain in a bag of sinomys of tk
            if ((s1[i] == s2[j] or s1[i] in sinH[j] or s2[j] in sinT[i]) and hash_s2[j] == 0) : 
                print(s1[i],s2[j],"sinonimos")
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s1[i] in hipH[j] or len((sinT[i]).intersection(hipH[j]))>0) and hash_s2[j] == 0):
                print("hiperonimos",s2[j],s1[i],(sinT[i]).intersection(hipH[j]))
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s2[j] in HipT[i] or len((sinH[j]).intersection(HipT[i]))>0) and hash_s2[j] == 0): 
                print("hiperonimos sobre sinonimos",s2[j],s1[i])
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif len((hipH[j]).intersection(HipT[i]))>0 and hash_s2[j] == 0: 
                print("hiph HiperT",s2[j],s1[i],hipH[j],HipT[i],(hipH[j]).intersection(HipT[i]))
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s1[i] in antH[j] or s2[j] in antT[i]) and hash_s2[j] == 0) : 
                print(s1[i],antH[j],s2[j],antT[i],"antonimos")
                hash_s1[i] -= 1; 
                hash_s2[j] -= 1; 
                matchC += 1; 
                break
            elif (len(HipH[j].intersection(HipT[i]))>0 and s1[i]!=s2[j] and hash_s2[j] == 0):
                print(s1[i],s2[j],HipH[j].intersection(HipT[i]),"antonimos")
                hash_s1[i] -= 1; 
                hash_s2[j] -= 1; 
                matchC += 1; 
                break
    print(hash_s1)
    print(hash_s2)
    print(match)
    print(matchC)
    # If there is no match 
    if (match == 0) :
        return 0.0,0.0,0.0; 
 
    # Number of transpositions 
    t = 0; 
 
    point = 0; 
 
    # Count number of occurrences 
    # where two characters match but 
    # there is a third matched character 
    # in between the indices 
    for i in range(len1) : 
        if (hash_s1[i]) :
 
            # Find the next matched character 
            # in second string 
            while (hash_s2[point] == 0) :
                point += 1; 
 
            if (s1[i] != s2[point]) :
                point += 1
                t += 1
            else :
                point += 1
                 
    t /= 2; 
    print(match)
    #Return the Jaro Similarity 
    #print(match / len2,matchC /len2,(match - t) / match)
    return (match/ len1 + match / len2 +
            (match - t) / match)/ 3.0; 


In [9]:
def jaro_distanceDavid2(s1, s2,sinT,sinH,antT,antH,HipT,HipH,hipH) :
    bandera=True

    # Length of two strings
    len1 = len(s1)
    len2 = len(s2)

    # If the listas de tokens are equal 
    if len1==len2:
        for i in range(len1):
            if s1[i]!=s2[i]:
                bandera=False
                break
        if (bandera):
            return 1.0,0.0; 
 
    if (len1 == 0 or len2 == 0) :
        return 0.0,0.0,0.0; 
 
    # Maximum distance upto which matching 
    # is allowed 
    max_dist = (max(len(s1), len(s2)) // 2 )-1 ; 
 
    # Count of matches 
    match = 0; 
    matchC = 0; 
 
    # Hash for matches 
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)
 
    # Traverse through the first string 
    for j in range(len2) : 
        # Check if there is any matches 
        for i in range(len1) : 
            print(s1[i],s2[j])
            # If there is a match or is contain in a bag of sinomys of tk
            if ((s1[i] == s2[j] or s1[i] in sinH[j] or s2[j] in sinT[i]) and hash_s2[j] == 0) : 
                print(s1[i],s2[j],"sinonimos")
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s1[i] in hipH[j] or len((sinT[i]).intersection(hipH[j]))>0) and hash_s2[j] == 0):
                print("hiperonimos",s2[j],s1[i],(sinT[i]).intersection(hipH[j]))
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s2[j] in HipT[i] or len((sinH[j]).intersection(HipT[i]))>0) and hash_s2[j] == 0): 
                print("hiperonimos sobre sinonimos",s2[j],s1[i])
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif len((hipH[j]).intersection(HipT[i]))>0 and hash_s2[j] == 0: 
                print("hiph HiperT",s2[j],s1[i],hipH[j],HipT[i],(hipH[j]).intersection(HipT[i]))
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s1[i] in antH[j] or s2[j] in antT[i]) and hash_s2[j] == 0) : 
                print(s1[i],antH[j],s2[j],antT[i],"antonimos")
                hash_s1[i] -= 1; 
                hash_s2[j] -= 1; 
                matchC += 1; 
                break
            elif (len(HipH[j].intersection(HipT[i]))>0 and s1[i]!=s2[j] and hash_s2[j] == 0):
                print(s1[i],s2[j],HipH[j].intersection(HipT[i]),"antonimos")
                hash_s1[i] -= 1; 
                hash_s2[j] -= 1; 
                matchC += 1; 
                break
    print(hash_s1)
    print(hash_s2)
    print(len2)
    print(match)
    print(matchC)
    # If there is no match 
    if (match == 0) :
        return 0.0,0.0,0.0; 
    return (match - matchC) /len2 ,(len2-match-matchC)/len2; 

In [10]:
# entailment
#"Police in Rio de Janeiro arrested five men and recovered millions of dollars worth of art stolen earlier this month, including works by Salvador Dali and Henri Matisse. Police recovered all the stolen art except two Chinese ceramic sculptures from the 7th Century and a collection of silverware.","Millions of dollars of art were recovered, including works by Dali.",YES
#Fishermen using the poison sodium cyanide have helped to destroy an estimated 70% of the reefs.,Cyanide fishing is linked to the destruction of area reefs.,YES
#Mr Ferdinand Piech said that the German company did not have any confidential plans or documents about GM's ultra-low-cost factory project.,Documents included secret plans for a new factory.,NO
#PM tried to buy the Belin biscuit company from RJR Nabisco two years ago.,American tobacco companies began to diversify production.,UNKNOWN
texto="Fishermen using poison sodium cyanide have helped destroy estimated 70% reefs"
hipotesis="Cyanide fishing linked destruction area reefs"

In [11]:
s1=texto.lower().split()
s2=hipotesis.lower().split()
sinT=[]
antT=[]
HipT=[]
sinH=[]
antH=[]
HipH=[]
hipH=[]

# encontrar bolsa de sinonimos de cada token
for t in s1:
    sinT.append(bag_of_synonyms(t))
    # antT.append(bag_of_antonyms(t))
    # HipT.append(bag_of_hyperonyms(t))

for h in s2:
    sinH.append(bag_of_synonyms(h))
    # antH.append(bag_of_antonyms(h))
    # HipH.append(bag_of_hyperonyms(h))
    # hipH.append(bag_of_hyponyms(h))
# sobre los sinonimos encontrar sus hiperonimos,hiponimos y antonimos
for k_t in range(len(sinT)):
    tempSet=set()
    tempSett=set()
    for s_ in sinT[k_t]:
        tempSet=tempSet.union(bag_of_antonyms(s_))
        tempSett=tempSett.union(bag_of_hyperonyms(s_))
    antT.append(tempSet)
    HipT.append(tempSett)
for k_t in range(len(sinH)):
    tempSet=set()
    tempSett=set()
    tempSettt=set()
    for s_ in sinH[k_t]:
        tempSet=tempSet.union(bag_of_antonyms(s_))
        tempSett=tempSett.union(bag_of_hyperonyms(s_))
        tempSettt=tempSettt.union(bag_of_hyponyms(s_))
    antH.append(tempSet)
    HipH.append(tempSett)
    hipH.append(tempSettt)

derived_from aurocyanide
derived_from bicyanide
derived_from cyanidation
derived_from cyanidol
derived_from dicyanide
derived_from ferrocyanide
derived_from hydrocyanide
derived_from isocyanide
derived_from leucocyanide
derived_from platinocyanide
derived_from protocyanide
derived_from selenocyanide
derived_from thiocyanide
derived_from tricyanide
form_of cyanided
form_of cyanides
form_of cyaniding
is_a potassium_cyanide
derived_from ferricyanide
derived_from ferrocyanide
is_a cyanide_gas
is_a cyanogen_chloride
is_a hydrocyanic_acid
is_a liquid_cyanide
is_a sodium_cyanide
is_a potassium_cyanide
is_a sodium_cyanide
derived_from acetonitrile
derived_from acrylonitrile
derived_from nitrile_rubber
derived_from phosphonitrile
derived_from aminonitrile
derived_from benzonitrile
derived_from carbonitrile
derived_from dinitrile
derived_from glutaronitrile
derived_from isonitrile
derived_from nitrilase
derived_from nitrilate
derived_from nitrilium
derived_from phosphonitrile
derived_from pipero

In [9]:
s1=texto.lower().split()
s2=hipotesis.lower().split()
sinT1=[]
antT1=[]
HipT1=[]
sinH1=[]
antH1=[]
HipH1=[]
hipH1=[]

# encontrar bolsa de sinonimos de cada token
for t in s1:
    sinT1.append(bag_of_synonyms(t))
    antT1.append(bag_of_antonyms(t))
    HipT1.append(bag_of_hyperonyms(t))

for h in s2:
    sinH1.append(bag_of_synonyms(h))
    antH1.append(bag_of_antonyms(h))
    HipH1.append(bag_of_hyperonyms(h))
    hipH1.append(bag_of_hyponyms(h))

derived_from aurocyanide
derived_from bicyanide
derived_from cyanidation
derived_from cyanidol
derived_from dicyanide
derived_from ferrocyanide
derived_from hydrocyanide
derived_from isocyanide
derived_from leucocyanide
derived_from platinocyanide
derived_from protocyanide
derived_from selenocyanide
derived_from thiocyanide
derived_from tricyanide
form_of cyanided
form_of cyanides
form_of cyaniding
is_a potassium_cyanide
derived_from ferricyanide
derived_from ferrocyanide
is_a cyanide_gas
is_a cyanogen_chloride
is_a hydrocyanic_acid
is_a liquid_cyanide
is_a sodium_cyanide
is_a potassium_cyanide
is_a sodium_cyanide
derived_from antifishing
derived_from electrofishing
derived_from fishing_line
derived_from nonfishing
derived_from rodfishing
derived_from shellfishing
derived_from spearfishing
derived_from sportfishing
derived_from sportsfishing
form_of fishings
has_prerequisite fishesing
has_subevent go_on_vacation
is_a fly_fishing
is_a angling
is_a angling
is_a bowfishing
is_a commercial

In [10]:
jaro_distanceDavid2(texto.lower().split(),hipotesis.lower().split(),sinT,sinH,antT,antH,HipT,HipH,hipH)

fishermen cyanide
using cyanide
poison cyanide
poison cyanide {'inanimate_object'} antonimos
fishermen fishing
using fishing
poison fishing
sodium fishing
cyanide fishing
have fishing
have fishing {'drink', 'getting_food', 'think'} antonimos
fishermen linked
using linked
poison linked
sodium linked
cyanide linked
have linked
helped linked
destroy linked
estimated linked
70% linked
reefs linked
fishermen destruction
using destruction
poison destruction
hiph HiperT destruction poison {'yardarm', 'revolutionize', 'finish', 'demolitions', 'kill', 'endling', 'endboard', 'closure', 'endorrhiz', 'yearend', 'coend', 'overthrowest', 'endplay', 'endomitosis', 'endodynamomorphic', 'endgame', 'endotherm', 'passing', 'terminus', 'endknot', 'wipe_out', 'endocrane', 'endostracum', 'endosymbiont', 'endochondral', 'fin_de_siecle', 'ender', 'self_destruction', 'devastating', 'homestretch', 'endbud', 'wipeouts', 'stub_out', 'endocuticle', 'endoplast', 'endsheet', 'endocyclic', 'endopsychic', 'endplate', 

(-0.16666666666666666, 0.16666666666666666)

In [11]:
jaro_distanceDavid2(texto.lower().split(),hipotesis.lower().split(),sinT1,sinH1,antT1,antH1,HipT1,HipH1,hipH1)

fishermen cyanide
using cyanide
poison cyanide
sodium cyanide
cyanide cyanide
cyanide cyanide sinonimos
fishermen fishing
using fishing
poison fishing
sodium fishing
cyanide fishing
have fishing
helped fishing
destroy fishing
estimated fishing
70% fishing
reefs fishing
fishermen linked
using linked
poison linked
sodium linked
cyanide linked
have linked
helped linked
destroy linked
estimated linked
70% linked
reefs linked
fishermen destruction
using destruction
poison destruction
hiph HiperT destruction poison {'wrack', 'fighting_war', 'kill', 'destructive', 'destructionist', 'photodestruction', 'ravage', 'fight_war', 'starting_fire', 'nondestruction', 'annihilation', 'eradication', 'having_party', 'ruin', 'holocaust', 'explosion', 'neutralization', 'spoliation', 'destructible', 'extermination', 'laying_waste', 'demolition', 'war', 'autodestruction', 'sabotage', 'destructions', 'disaster', 'cyclodestruction', 'angiodestruction', 'decimation', 'thunderstorm', 'self_destruction', 'vandali

(0.5, 0.5)

# Implementación de Jaro-Winkler

In [1]:
# Python3 implementation of above approach 
from math import floor
 
# Function to calculate the 
# Jaro Similarity of two strings 
def jaro_distanceO(s1, s2,sinT,sinH,antT,antH,HipT,HipH,hipH) :
    bandera=True

    # Length of two strings
    len1 = len(s1)
    len2 = len(s2)

    # If the listas de tokens are equal 
    if len1==len2:
        for i in range(len1):
            if s1[i]!=s2[i]:
                bandera=False
                break
        if (bandera):
            return 1.0; 
 
    if (len1 == 0 or len2 == 0) :
        return 0.0; 
    
    # Maximum distance upto which matching 
    # is allowed 
    max_dist = (max(len(s1), len(s2)) // 2 ) - 1; 
 
    # Count of matches 
    match = 0; 
 
    # Hash for matches 
    hash_s1 = [0] * len(s1) ;
    hash_s2 = [0] * len(s2) ; 
 
    # Traverse through the first string 
    for i in range(len1) : 
 
        # Check if there is any matches 
        for j in range( max(0, i - max_dist), 
                    min(len2, i + max_dist + 1)) : 
             
            # If there is a match 
            if (s1[i] == s2[j] and hash_s2[j] == 0) : 
                hash_s1[i] = 1; 
                hash_s2[j] = 1; 
                match += 1; 
                break; 
         
    # If there is no match 
    if (match == 0) :
        return 0.0; 
    print(hash_s1)
    print(hash_s2)
    # Number of transpositions 
    t = 0; 
 
    point = 0; 
 
    # Count number of occurrences 
    # where two characters match but 
    # there is a third matched character 
    # in between the indices 
    for i in range(len1) : 
        if (hash_s1[i]) :
 
            # Find the next matched character 
            # in second string 
            while (hash_s2[point] == 0) :
                point += 1; 
 
            if (s1[i] != s2[point]) :
                point += 1;
                t += 1;
            else :
                point += 1;
                 
    t /= 2; 
    print(match)
    # Return the Jaro Similarity 
    return (match/ len1 + match / len2 +
            (match - t) / match)/ 3.0; 
 
# Jaro Winkler Similarity 
def jaro_WinklerO(s1, s2) : 
 
    jaro_dist = jaro_distanceO(s1, s2); 
 
    # If the jaro Similarity is above a threshold 
    if (jaro_dist > 0.7) :
 
        # Find the length of common prefix 
        prefix = 0; 
 
        for i in range(min(len(s1), len(s2))) :
         
            # If the characters match 
            if (s1[i] == s2[i]) :
                prefix += 1; 
 
            # Else break 
            else :
                break; 
 
        # Maximum of 4 characters are allowed in prefix 
        prefix = min(4, prefix); 
 
        # Calculate jaro winkler Similarity 
        jaro_dist += 0.1 * prefix * (1 - jaro_dist); 
 
    return jaro_dist; 

In [2]:
# Python3 implementation of above approach
from math import floor, ceil
 
# Function to calculate the
# Jaro Similarity of two s
def jaro_distance(s1, s2):
     
    # If the s are equal
    if (s1 == s2):
        return 1.0
 
    # Length of two s
    len1 = len(s1)
    len2 = len(s2)
 
    # Maximum distance upto which matching
    # is allowed
    max_dist = floor(max(len1, len2) / 2) - 1
 
    # Count of matches
    match = 0
 
    # Hash for matches
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)
 
    # Traverse through the first
    for i in range(len1):
 
        # Check if there is any matches
        for j in range(max(0, i - max_dist), 
                       min(len2, i + max_dist + 1)):
             
            # If there is a match
            if (s1[i] == s2[j] and hash_s2[j] == 0):
                hash_s1[i] = 1
                hash_s2[j] = 1
                match += 1
                break
 
    # If there is no match
    if (match == 0):
        return 0.0
 
    # Number of transpositions
    t = 0
    point = 0
 
    # Count number of occurrences
    # where two characters match but
    # there is a third matched character
    # in between the indices
    for i in range(len1):
        if (hash_s1[i]):
 
            # Find the next matched character
            # in second
            while (hash_s2[point] == 0):
                point += 1
 
            if (s1[i] != s2[point]):
                t += 1
            point += 1
    t = t//2
 
    # Return the Jaro Similarity
    return (match/ len1 + match / len2 +
            (match - t) / match)/ 3.0
 
# Driver code
s1 = "arnab"
s2 = "raanb"
 
# Prjaro Similarity of two s
print(round(jaro_distance(s1, s2),6))
 
# This code is contributed by mohit kumar 29

0.866667


In [2]:
jaro_distanceO(texto.split(),hipotesis.split())

NameError: name 'texto' is not defined

In [14]:
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_md")
##"The sale was made to pay Yukos' US$ 27.5 billion tax bill, Yuganskneftegaz was originally sold for US$ 9.4 billion to a little known company Baikalfinansgroup which was later bought by the Russian state-owned oil company Rosneft .",Baikalfinansgroup was sold to Rosneft.,YES
##
# Process whole documents
text = ("Following the Declaration of the Establishment of the State of Israel, May 14, 1948, seven Arab states entered Palestine and engaged Israeli forces.")
doc = nlp(text)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

/home/david/anaconda3/envs/rit/lib/python3.9/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Noun phrases: ['the Declaration', 'the Establishment', 'the State', 'Israel', 'seven Arab states', 'Palestine', 'Israeli forces']
Verbs: ['follow', 'enter', 'engage']
the State of Israel GPE
May 14, 1948 DATE
seven CARDINAL
Arab NORP
Palestine GPE
Israeli NORP


In [15]:
# Process whole documents
text = ("Israeli forces attacked seven Arab states in 1948.")
doc = nlp(text)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

Noun phrases: ['Israeli forces', 'seven Arab states']
Verbs: ['attack']
Israeli NORP
seven CARDINAL
Arab NORP
1948 DATE


In [16]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

Israeli israeli ADJ JJ amod Xxxxx True False
forces force NOUN NNS nsubj xxxx True False
attacked attack VERB VBD ROOT xxxx True False
seven seven NUM CD nummod xxxx True False
Arab arab ADJ JJ amod Xxxx True False
states state NOUN NNS dobj xxxx True False
in in ADP IN prep xx True True
1948 1948 NUM CD pobj dddd False False
. . PUNCT . punct . False False


In [17]:
def jaro_distance_prueba(s1, s2) :
    sinT=[]
    sinH=[]
    HipT=[]
    hipH=[]
    HipH=[]
    antT=[]
    antH=[]
    for t in s1:
        sinT.append(bag_of_synonyms(t))
    for h in s2:
        sinH.append(bag_of_synonyms(h))
    
    tempSet=set()
    tempSet1=set()
    for i in range(len(sinT)):
        tempSet=set()
        tempSet1=set()
        for s_ in sinT[i]:
            tempSet=tempSet.union(bag_of_hyperonyms(s_))
            tempSet1=tempSet1.union(bag_of_antonyms(s_))
        antT.append(tempSet1)
        HipT.append(tempSet)
    for i in range(len(sinH)):
        tempSet=set()
        tempSet1=set()
        tempSett2=set()
        for s_ in sinH[i]:
            tempSet=tempSet.union(bag_of_hyponyms(s_))
            tempSet1=tempSet1.union(bag_of_antonyms(s_))
            tempSett2=tempSett2.union(bag_of_hyperonyms(s_))
        antH.append(tempSet1)
        HipH.append(tempSett2)
        hipH.append(tempSet)
            
        
    print("sinonimos de T",sinT)
    print("Hiperonimos de T",HipT)
    print("sinonimos de H",sinH)
    print("hiponimos de h",hipH)

    bandera=True

    # Length of two strings
    len1 = len(s1)
    len2 = len(s2)

    # If the listas de tokens are equal 
    if len1==len2:
        for i in range(len1):
            if s1[i]!=s2[i]:
                bandera=False
                break
        if (bandera):
            return 1.0,0.0; 
 
    if (len1 == 0 or len2 == 0) :
        return 0.0,0.0; 
 
    # Maximum distance upto which matching 
    # is allowed 
    max_dist = (max(len(s1), len(s2)) // 2 )-1 ; 
 
    # Count of matches 
    match = 0; 
    match1 = 0; 
 
    # Hash for matches 
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)
 
    # Traverse through the first string 
    for i in range(len1) : 
            
        # Check if there is any matches 
        for j in range( max(0, i - max_dist),
                    min(len2, i + max_dist + 1)) : 
            print(s1[i],s2[j])
            # If there is a match or is contain in a bag of sinomys of tk
            if ((s1[i] == s2[j] or s1[i] in sinH[j] or s2[j] in sinT[i]) and hash_s2[j] == 0) : 
                print(s1[i],s2[j],"sinonimos")
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s1[i] in hipH[j] or len((sinT[i]).intersection(hipH[j]))>0) and hash_s2[j] == 0):
                print("hiperonimos",s2[j],s1[i],(sinT[i]).intersection(hipH[j]))
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s2[j] in HipT[i] or len((sinH[j]).intersection(HipT[i]))>0) and hash_s2[j] == 0): 
                print("hiperonimos sobre sinonimos",s2[j],s1[i])
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif len((hipH[j]).intersection(HipT[i]))>0 and hash_s2[j] == 0: 
                print("hiperonimos3",s2[j],s1[i],(hipH[j]).intersection(HipT[i]))
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s1[i] in antH[j] or s2[j] in antT[i]) and hash_s2[j] == 0) : 
                print(s1[i],antH[j],s2[j],antT[i],"antonimos")
                hash_s1[i] = 2; 
                hash_s2[j] = 2; 
                match1 += 1; 
                break
            elif (len(HipH[j].intersection(HipT[i]))>0 and s1[i]!=s2[j] and hash_s2[j] == 0):
                print(s1[i],s2[j],HipH[j].intersection(HipT[i]),"antonimos")
                # print(HipT[i])
                # print(HipH[j])
                hash_s1[i] = 2; 
                hash_s2[j] = 2; 
                match1 += 1; 
                break
    print(hash_s1)
    print(hash_s2)
    print("inferencias",match)
    print("contradicciones",match1)
    # If there is no match 
    if (match == 0) :
        return 0.0,0.0; 
 
    # Number of transpositions 
    t = 0; 
 
    point = 0; 
 
    # Count number of occurrences 
    # where two characters match but 
    # there is a third matched character 
    # in between the indices 
    for i in range(len1) : 
        if (hash_s1[i]) :
 
            # Find the next matched character 
            # in second string 
            while (hash_s2[point] == 0) :
                point += 1; 
 
            if (s1[i] != s2[point]) :
                point += 1
                t += 1
            else :
                point += 1
                 
    t /= 2; 
    print(t)
    #Return the Jaro Similarity 
    return ((match / len2) - (match1 /len2) +
            (match - t) / match ) / 3.0; 

In [18]:
texto="Swansea striker Lee Trundle has negotiated a lucrative image-rights deal with the League One club"
hipotesis="Lee Trundle is in business with the League One club"

In [19]:
jaro_distance_prueba(texto.lower().split(),hipotesis.lower().split())

derived_from downwinder
derived_from alee
derived_from annalee
derived_from emmalee
derived_from leeboard
derived_from leeside
derived_from leeward
derived_from leewards
derived_from leeway
form_of lees
derived_from lee_county
derived_from robert_lee
derived_from leeward_islands
derived_from leewardness
form_of leewardmost
is_a leeward
form_of trundle_beds
form_of lantern_wheels
derived_from trundle_out
derived_from trundlehead
derived_from trundler
derived_from trundletail
form_of trundled
form_of trundles
form_of trundles
form_of trundling
form_of trunnel
derived_from isisism
form_of isises
derived_from is
causes expressing_how_funny
derived_from as_is
derived_from isentropic
derived_from isn't
derived_from isness
derived_from tiswas
form_of ees
form_of is_as
form_of isint
form_of sn’t
derived_from copper_indium_gallium_selenide
derived_from indic
derived_from indinic
derived_from indium_ammonium_alum
derived_from indium_antimonide
derived_from indium_arsenide
derived_from indium_bro

0.2857142857142857

# reformar sentence

In [20]:
import utils as ut
import spacy
nlp = spacy.load("en_core_web_md") # modelo de nlp
ut.load_vectors_in_lang(nlp,"./data/numberbatch-en-17.04b.txt") # carga de vectores en nlp.wv

In [21]:
def reform_sentence2(frase,nlp):
    doc = nlp(" ".join(frase))
    poses=[]
    lemas=[]
    for token in doc:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #    token.shape_, token.is_alpha, token.is_stop)
        poses.append(token.pos_)
        lemas.append(token.lemma_)
    frase.append("")

    frase_m=[]
    b=1
    i=-1
    bandera=True
    while bandera==True:
        i+=1
        if i>=len(frase)-3:
            frase_m.append(frase[i])
            frase_m.append(frase[i+1])
            break
        word_h1=frase[i]+"_"+frase[i+1]
        word_h2=frase[i]+"_"+frase[i+2]
        try:
            if word_h1 in nlp.wv:
                #frase_m.append(frase[i])
                frase_m.append(word_h1)
                print(word_h1,poses[i],poses[i+1],lemas[i]+"_"+lemas[i+1])
                b=0
                i+=1
            elif word_h2 in nlp.wv:
                frase_m.append(word_h2)
                print(word_h2,poses[i],poses[i+2],lemas[i]+"_"+lemas[i+2])
                b=0
                i+=2
            elif b==0:
                frase_m.append(frase[i])
                b=1
            else:
                frase_m.append(frase[i])
        except:
            pass
    text = " ".join(frase_m)
    return text

In [22]:
#A young model has her first photoshoot
frase="A person taking pictures of a young brunette girl a"

In [23]:
reform_sentence2(frase.split(),nlp)

taking_pictures VERB NOUN take_picture
young_girl ADJ NOUN young_girl


'A person taking_pictures of a young_girl a '

In [24]:
reform_sentence2("A young model has her first photoshoot".split(),nlp)

'A young model has her first photoshoot'

In [25]:
frase1="Female mosquitoes become infected with the malaria parasite when they draw blood from humans with malaria. The insects can then pass this on to other humans they bite, but do not get sick themselves."

In [26]:
reform_sentence2(frase1.split(),nlp)

draw_blood VERB NOUN draw_blood
pass_on ADV PRON then_this
do_not PUNCT CCONJ ,_but
get_sick AUX PART do_not


'Female mosquitoes become infected with the malaria parasite when they draw_blood from humans with malaria. The insects can then pass_on to other humans they bite, but do_not get_sick themselves. '

In [27]:
doc = nlp(frase)
anterior=""
pos_ant=""
for token in doc:
    if token.pos_ in ["NOUN","VERB","ADJ","ADV","PROPN","NUM"]:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #    token.shape_, token.is_alpha, token.is_stop)
        print(anterior,pos_ant,token.text,token.pos_)
        anterior=token.text
        pos_ant=token.pos_
        #for e in token.ancestors:
        #    print(token.text,e,e.pos_)
    elif token.text==".":
        anterior=""
        pos_ant=""

  person NOUN
person NOUN taking VERB
taking VERB pictures NOUN
pictures NOUN young ADJ
young ADJ brunette NOUN
brunette NOUN girl NOUN


In [86]:
f="Female mosquitoes become infected with the malaria parasite when they draw blood from humans with malaria. The insects can then pass on to other humans they bite, but do not get sick themselves."
f="A young model has her first photoshoot"
#f="A person taking pictures of a young brunette girl"
#f="several younger people sitting in front of a statue"
#f="Gold mining operations in California and Nevada use cyanide to extract the precious metal."
#f="several young people are sitting in an auditorium"
#f="Cyanide is used in gold mining."
#f="A stuffed animal on the couch."
#f="An animal on the couch."
#f="State police investigators testified at a preliminary hearing that Delbert told them the death of the 64-year-old Billy was a mercy killing"
# f="Two women are embracing while holding to go packages."
# f="The sisters are hugging goodbye while holding to go packages after just eating lunch."
#f="Two young children in blue jerseys, one with the number 9 and one with the number 2 are standing on wooden steps in a bathroom and washing their hands in a sink."
# f="Two kids in numbered jerseys wash their hands."
#f="A woman is doing a cartwheel while wearing a bikini in the sand next to the beach."
#f="A woman is doing a cartwheel"
# f="A senior is waiting at the window of a restaurant that serves sandwiches."
# f="A person waits to be served his food."
#f="A man wearing a blue shirt, black apron, eyeglasses, and a mask is handling food in a tray."
#f="A person in appropriate institution protection works with meals."
#f="A man dressed in a red shirt and black tie stands up at a wedding reception to make a speech."
#f="a guy in a red top and tie makes a speech"

#f="known company Baikalfinansgroup which was later bought by the Russian state-owned oil company Rosneft ."
#f="Baikalfinansgroup was sold to Rosneft."
#f="The sale was made to pay Yukos' US$ 27.5 billion tax bill, Yuganskneftegaz was originally sold for US$9.4 billion to a little known company Baikalfinansgroup which was later bought by the Russian state-owned oil company Rosneft ."
#f="Yuganskneftegaz cost US$ 27.5 billion."
#f="Kids ride an amusement ride."
#f="Fishermen using poison sodium cyanide have helped destroy estimated 70% reefs"
#f="Cyanide fishing linked destruction area reefs"
#f="A stuffed animal on the couch."
#f="A young boy runs across a road in front of a sky blue building with barred windows."
#f="Autonomous cars shift insurance liability toward manufacturers"
f="Mohammad Khalifa's father_in_law is Osama Bin Laden"
#f="The brother_in_law of Mohammad Oreibi al-Khalifa the new chief judge in the genocide trial of Saddam Hussein has been shot dead police sources say"
f="The sale was made to pay Yukos' US$ 27.5 billion tax bill, Yuganskneftegaz was originally sold for US$9.4 billion to a little known company Baikalfinansgroup which was later bought by the Russian state-owned oil company Rosneft ."

In [87]:
doc = nlp(f)
poses=[]
tokens=[]
lemmas=[]
for token in doc:
    poses.append(token.pos_)
    tokens.append(token.text)
    lemmas.append(token.lemma_)
    # if token.pos_ in ["NOUN","VERB","ADJ","ADV","PRON","PART","PROPN","ADP","NUM"] or token.dep_=="neg":
    #     poses.append(token.pos_)
    #     tokens.append(token.text)
    #     lemmas.append(token.lemma_)
    # elif token.text ==".":
    #     poses.append(token.pos_)
    #     tokens.append(token.text)
    #     lemmas.append(token.lemma_)
    # else:
    #     print(token.text,token.pos_,token.dep_)
poses.append("<F>")
tokens.append("</F>")
lemmas.append("</F>")
print(tokens)
print(poses)
print(lemmas)

['The', 'sale', 'was', 'made', 'to', 'pay', 'Yukos', "'", 'US$', '27.5', 'billion', 'tax', 'bill', ',', 'Yuganskneftegaz', 'was', 'originally', 'sold', 'for', 'US$', '9.4', 'billion', 'to', 'a', 'little', 'known', 'company', 'Baikalfinansgroup', 'which', 'was', 'later', 'bought', 'by', 'the', 'Russian', 'state', '-', 'owned', 'oil', 'company', 'Rosneft', '.', '</F>']
['DET', 'NOUN', 'AUX', 'VERB', 'PART', 'VERB', 'PROPN', 'PUNCT', 'SYM', 'NUM', 'NUM', 'NOUN', 'NOUN', 'PUNCT', 'PROPN', 'AUX', 'ADV', 'VERB', 'ADP', 'SYM', 'NUM', 'NUM', 'ADP', 'DET', 'ADJ', 'VERB', 'NOUN', 'PROPN', 'PRON', 'AUX', 'ADV', 'VERB', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'VERB', 'NOUN', 'NOUN', 'PROPN', 'PUNCT', '<F>']
['the', 'sale', 'be', 'make', 'to', 'pay', 'Yukos', "'", 'us$', '27.5', 'billion', 'tax', 'bill', ',', 'Yuganskneftegaz', 'be', 'originally', 'sell', 'for', 'us$', '9.4', 'billion', 'to', 'a', 'little', 'know', 'company', 'Baikalfinansgroup', 'which', 'be', 'later', 'buy', 'by', 'the', 'russian',

In [88]:
i=0
while i <=(len(tokens)-1):
    if poses[i]=="ADJ" and poses[i+1]=="NOUN" and poses[i+2]=="NOUN":
        print(tokens[i],tokens[i+2])
        print(tokens[i+1],tokens[i+2])
        i+=2
    elif poses[i]=="ADJ" and (poses[i+1]=="NOUN" or poses[i+1]=="PRON"):
        print(tokens[i],tokens[i+1])
        i+=1
    elif poses[i]=="ADJ" and poses[i+1]=="ADJ" and poses[i+2]=="NOUN":
        print(tokens[i],tokens[i+2])
        print(tokens[i+1],tokens[i+2])
        i+=2
    elif poses[i]=="ADV" and poses[i+1]=="VERB":
        print(tokens[i],tokens[i+1])
        i+=1
    # elif poses[i]=="VERB" and poses[i+1]=="NOUN":
    #     print(tokens[i],tokens[i+1])
    #     i+=1
    elif poses[i]=="VERB":
        print(tokens[i])
    elif poses[i]=="NOUN" and poses[i+1]=="NOUN":
        print(tokens[i+1],tokens[i])
        i+=1
    elif poses[i]=="NOUN" or poses[i]=="PRON" or poses[i]=="PROPN":
        print(tokens[i],"w")
    elif poses[i]=="PART":
        print(tokens[i])
    elif poses[i]=="ADP":
         print(tokens[i])
    elif poses[i]=="SYM" and poses[i+1]=="NUM":
        print(tokens[i],tokens[i+1])
        i+=1
    elif poses[i]=="NUM" and poses[i+1]=="SYM":
        print(tokens[i],tokens[i+1])
        i+=1
    elif poses[i]=="NUM" or poses[i]=="SYM":
        print(tokens[i])
    else:
        print()
    i+=1


sale w

made
to
pay
Yukos w

US$ 27.5
billion
bill tax

Yuganskneftegaz w

originally sold
for
US$ 9.4
billion
to


known
company w
Baikalfinansgroup w
which w

later bought
by

Russian state

owned
company oil
Rosneft w




In [89]:
f

"The sale was made to pay Yukos' US$ 27.5 billion tax bill, Yuganskneftegaz was originally sold for US$9.4 billion to a little known company Baikalfinansgroup which was later bought by the Russian state-owned oil company Rosneft ."

In [90]:
doc = nlp(f)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)
for token in doc:
    if token.text in ["cars","manufacturers","Autonomous"]:
        for e in token.ancestors:
            print(token.text,e,e.pos_)
    elif token.text=="liability":
        for e in token.ancestors:
            print(token.text,e,e.pos_)

Noun phrases: ['The sale', "Yukos' US$ 27.5 billion tax bill", 'Yuganskneftegaz', 'a little known company', 'which', 'the Russian state-owned oil company', 'Rosneft']
Verbs: ['make', 'pay', 'sell', 'know', 'buy', 'own']
Yukos ORG
US$ 27.5 billion MONEY
Yuganskneftegaz ORG
US$9.4 billion MONEY
Baikalfinansgroup ORG
Russian NORP
Rosneft ORG


In [91]:
doc = nlp(f)
anterior=""
pos_ant=""
for token in doc:
    if token.pos_ in ["NOUN","VERB","ADJ","ADV","PROPN","NUM"]:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #    token.shape_, token.is_alpha, token.is_stop)
        print(anterior,pos_ant,token.text,token.pos_)
        anterior=token.text
        pos_ant=token.pos_
        #for e in token.ancestors:
        #    print(token.text,e,e.pos_)
    elif token.text==".":
        anterior=""
        pos_ant=""

  sale NOUN
sale NOUN made VERB
made VERB pay VERB
pay VERB Yukos PROPN
Yukos PROPN 27.5 NUM
27.5 NUM billion NUM
billion NUM tax NOUN
tax NOUN bill NOUN
bill NOUN Yuganskneftegaz PROPN
Yuganskneftegaz PROPN originally ADV
originally ADV sold VERB
sold VERB 9.4 NUM
9.4 NUM billion NUM
billion NUM little ADJ
little ADJ known VERB
known VERB company NOUN
company NOUN Baikalfinansgroup PROPN
Baikalfinansgroup PROPN later ADV
later ADV bought VERB
bought VERB Russian ADJ
Russian ADJ state NOUN
state NOUN owned VERB
owned VERB oil NOUN
oil NOUN company NOUN
company NOUN Rosneft PROPN


In [92]:
import spacy

nlp = spacy.load("en_core_web_md")
doc = nlp(f)
for token in doc: #revisar esta dbido a que vincula los verbos para hacer el procesamiento
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])


The det sale NOUN []
sale nsubjpass made VERB [The]
was auxpass made VERB []
made ccomp sold VERB [sale, was, pay]
to aux pay VERB []
pay advcl made VERB [to, bill]
Yukos poss bill NOUN [']
' punct Yukos PROPN []
US$ quantmod billion NUM []
27.5 compound billion NUM []
billion nummod bill NOUN [US$, 27.5]
tax compound bill NOUN []
bill dobj pay VERB [Yukos, billion, tax]
, punct sold VERB []
Yuganskneftegaz nsubjpass sold VERB []
was auxpass sold VERB []
originally advmod sold VERB []
sold ROOT sold VERB [made, ,, Yuganskneftegaz, was, originally, for, to, Baikalfinansgroup, .]
for prep sold VERB [billion]
US$ quantmod billion NUM []
9.4 compound billion NUM []
billion pobj for ADP [US$, 9.4]
to prep sold VERB [company]
a det company NOUN []
little advmod known VERB []
known amod company NOUN [little]
company pobj to ADP [a, known]
Baikalfinansgroup dep sold VERB [bought]
which nsubjpass bought VERB []
was auxpass bought VERB []
later advmod bought VERB []
bought relcl Baikalfinansgrou

In [93]:
import spacy
from spacy.symbols import nsubj, VERB

nlp = spacy.load("en_core_web_md")
doc = nlp(f)

# Finding a verb with a subject from below — good
verbs = set()
for possible_subject in doc:
    if possible_subject.dep == nsubj and possible_subject.head.pos == VERB:
        verbs.add(possible_subject.head)
        print(possible_subject)
        for t in possible_subject.children:
            print(t)
        
print(verbs)

set()


## otro

 

In [94]:
doc = nlp(f)
anterior=""
pos_ant=""
adjetivos=[]
hayADJ=False
adverbios=[]
hayADV=False
sustantivos = []
hayNOUN=[]
for token in doc:
    if token.pos_ in ["NOUN","VERB","ADJ","ADV"]:
        if token.pos_ == "ADJ":
            adjetivos.append(token.text)
            hayADJ=True
        elif token.pos_ == "ADV":
            adverbios.append(token.text)
            hayADV=True
        elif token.pos_=="NOUN" and pos_ant=="NOUN":
            print(anterior,token.text)
        elif token.pos_ == "NOUN" and hayADJ:
            for a in adjetivos:
                print(a,token.text)
            hayADJ=False
            adjetivos=[]
        elif token.pos_ == "NOUN" and hayADJ==False:
            print(token.text,token.pos_)
        elif token.pos_ == "VERB" and hayADV:
            for a in adverbios:
                print(a,token.text,token.pos_)
            hayADV=False
            adverbios=[]
        elif token.pos_ == "VERB" and hayADV==False:
            print(token.text,token.pos_)
        else:
            #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            #    token.shape_, token.is_alpha, token.is_stop)
            print(anterior,pos_ant,token.text,token.pos_)
            anterior=token.text
            pos_ant=token.pos_
            #for e in token.ancestors:
            #    print(token.text,e,e.pos_)
        anterior=token.text
        pos_ant=token.pos_
    elif token.text==".":
        anterior=""
        pos_ant=""

sale NOUN
made VERB
pay VERB
tax NOUN
tax bill
originally sold VERB
known VERB
little company
later bought VERB
Russian state
owned VERB
oil NOUN
oil company


In [3]:
doc = nlp("several younger people sitting in front of a statue")
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        token.shape_, token.is_alpha, token.is_stop)
    for e in token.ancestors:
        print(token.text,e,e.pos_)

NameError: name 'nlp' is not defined

In [251]:
doc = nlp(f)
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)
    for e in token.children:
            print(e.pos_)

A a DET DT det X True True
person person NOUN NN nsubj xxxx True False
DET
waits wait VERB VBZ ROOT xxxx True False
NOUN
VERB
PUNCT
to to PART TO aux xx True True
be be AUX VB auxpass xx True True
served serve VERB VBN xcomp xxxx True False
PART
AUX
NOUN
his his PRON PRP$ poss xxx True True
food food NOUN NN dobj xxxx True False
PRON
. . PUNCT . punct . False False


In [95]:
import spacy
from spacy.lang.en import English

nlp = English()  # just the language with no pipeline
nlp.add_pipe("sentencizer")
doc = nlp("The sale was made to pay Yukos' US$ 27.5 billion tax bill, Yuganskneftegaz was originally sold for US$9.4 billion to a little known company Baikalfinansgroup which was later bought by the Russian state-owned oil company Rosneft .")
for sent in doc.sents:
    print(sent.text)


The sale was made to pay Yukos' US$ 27.5 billion tax bill, Yuganskneftegaz was originally sold for US$9.4 billion to a little known company Baikalfinansgroup which was later bought by the Russian state-owned oil company Rosneft .


In [79]:
from spacy.language import Language
import spacy

text = "this is a sentence...hello...and another sentence."

nlp = spacy.load("en_core_web_md")
doc = nlp(text)
print("Before:", [sent.text for sent in doc.sents])

@Language.component("set_custom_boundaries")
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if token.text == "...":
            doc[token.i + 1].is_sent_start = True
    return doc

nlp.add_pipe("set_custom_boundaries", before="parser")
doc = nlp(text)
print("After:", [sent.text for sent in doc.sents])

Before: ['this is a sentence...', 'hello...and another sentence.']
After: ['this is a sentence...', 'hello...', 'and another sentence.']


In [1]:
import pandas as pd

In [2]:
dt = pd.read_pickle("/home/david/RIT_abril2024/salida/pruebaSBS/RTE3/1/DEV_1.csv_.pickle")

In [4]:
dt.columns

Index(['sumas', 'distancias', 'entropia_total', 'entropias', 'mutinf',
       'mearts', 'max_info', 'similitud_faltantes', 'list_comp', 'list_incomp',
       'list_rel_con', 'list_M', 'list_m', 'list_T', 'list_relaciones',
       'listas_malign', 'listas_malignf', 'list_bigram', 'diferencias',
       'bi_ent_trip_t', 'bi_ent_trip_h', 'bi_ent_trip_rel', 'Jaro-Winkler_rit',
       'nomatch', 'entidades', 'clases'],
      dtype='object')

In [1]:
import numpy as np

In [7]:
np.zeros((1,300))[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.